# Example where block/coordinate descent doesn't reach the same solution

They should converge to the same solution, but in practice they don't, and this notebook illustrates such an example using a $10 \times 10$ matrix. 

In [1]:
# load packages needed for this tutorial
using Revise
using Knockoffs
using Random
using GLMNet
using Distributions
using LinearAlgebra
using ToeplitzMatrices
using StatsBase
using CSV, DataFrames
using Plots
gr(fmt=:png);

function get_sigma(option::Int, p::Int)
    # note: groups are defined empirically within each simuation
    datadir = "/Users/biona001/Benjamin_Folder/research/4th_project_PRS/group_knockoff_test_data"
    if option == 1
        ρ = 0.7
        Σ = SymmetricToeplitz(ρ.^(0:(p-1))) |> Matrix
    elseif option == 2
        ρ = 0.7
        γ = 0.1
        groups = repeat(1:Int(p/5), inner=5)
        Σ = simulate_block_covariance(groups, ρ, γ)
    elseif option == 3
        covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame) # 3782 SNPs
        Σ = covfile |> Matrix{Float64}
        Σ = 0.99Σ + 0.01I #ensure PSD
    elseif option == 4
        df = CSV.read(joinpath(datadir, "21_37870779_38711704.csv"), DataFrame)
        Σ = df[:, 7:end] |> Matrix |> Symmetric |> Matrix
    elseif option == 5
        df = CSV.read(joinpath(datadir, "22_17674295_18295575.csv"), DataFrame)
        Σ = df[:, 7:end] |> Matrix |> Symmetric |> Matrix
    else
        error("Option should be 1-5 but was $option")
    end
    return Σ[1:p, 1:p]
end

sigma_option = 2
p = 10
Σ = get_sigma(sigma_option, p)

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1423


10×10 Matrix{Float64}:
 1.0   0.7   0.7   0.7   0.7   0.07  0.07  0.07  0.07  0.07
 0.7   1.0   0.7   0.7   0.7   0.07  0.07  0.07  0.07  0.07
 0.7   0.7   1.0   0.7   0.7   0.07  0.07  0.07  0.07  0.07
 0.7   0.7   0.7   1.0   0.7   0.07  0.07  0.07  0.07  0.07
 0.7   0.7   0.7   0.7   1.0   0.07  0.07  0.07  0.07  0.07
 0.07  0.07  0.07  0.07  0.07  1.0   0.7   0.7   0.7   0.7
 0.07  0.07  0.07  0.07  0.07  0.7   1.0   0.7   0.7   0.7
 0.07  0.07  0.07  0.07  0.07  0.7   0.7   1.0   0.7   0.7
 0.07  0.07  0.07  0.07  0.07  0.7   0.7   0.7   1.0   0.7
 0.07  0.07  0.07  0.07  0.07  0.7   0.7   0.7   0.7   1.0

$\Sigma$ is 10 by 10 with obvious blocks. Lets define 2 groups that capture this structure.

In [2]:
groups = repeat(1:2, inner=5)
groups

10-element Vector{Int64}:
 1
 1
 1
 1
 1
 2
 2
 2
 2
 2

Lets solve for SDP knockoff using various methods

In [3]:
@time equi, _, _ = solve_s_group(Symmetric(Σ), groups, :equi)
@time sdp_subopt, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp_subopt)
@time sdp_block, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp_block)
@time sdp_ccd, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp, robust=false);

 19.908376 seconds (66.64 M allocations: 3.457 GiB, 5.60% gc time, 99.97% compilation time)
 34.539005 seconds (125.63 M allocations: 7.409 GiB, 3.62% gc time, 99.56% compilation time)
  1.936544 seconds (4.95 M allocations: 264.060 MiB, 1.77% gc time, 93.05% compilation time)


LoadError: UndefVarError: jj not defined

In [99]:
equi

10×10 Matrix{Float64}:
 1.0  0.7  0.7  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  1.0  0.7  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  1.0  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  0.7  1.0  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  0.7  0.7  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.7  0.7  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  1.0  0.7  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  1.0  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  0.7  1.0  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  0.7  0.7  1.0

In [100]:
sdp_subopt

10×10 Matrix{Float64}:
 1.0  0.7  0.7  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  1.0  0.7  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  1.0  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  0.7  1.0  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  0.7  0.7  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.7  0.7  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  1.0  0.7  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  1.0  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  0.7  1.0  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  0.7  0.7  1.0

In [101]:
sdp_ccd

10×10 Matrix{Float64}:
 0.999987  0.35      0.35      0.35      …  0.0       0.0       0.0
 0.35      0.869155  0.349986  0.34999      0.0       0.0       0.0
 0.35      0.349986  0.50042   0.350077     0.0       0.0       0.0
 0.35      0.34999   0.350077  0.500001     0.0       0.0       0.0
 0.35      0.349992  0.350038  0.350019     0.0       0.0       0.0
 0.0       0.0       0.0       0.0       …  0.349993  0.349996  0.349998
 0.0       0.0       0.0       0.0          0.350021  0.350011  0.350005
 0.0       0.0       0.0       0.0          0.500001  0.350003  0.350001
 0.0       0.0       0.0       0.0          0.350003  0.5       0.350001
 0.0       0.0       0.0       0.0          0.350001  0.350001  0.5

In [102]:
sdp_block

10×10 Matrix{Float64}:
 1.0  0.7  0.7  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  1.0  0.7  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  1.0  0.7  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  0.7  1.0  0.7  0.0  0.0  0.0  0.0  0.0
 0.7  0.7  0.7  0.7  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.7  0.7  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  1.0  0.7  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  1.0  0.7  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  0.7  1.0  0.7
 0.0  0.0  0.0  0.0  0.0  0.7  0.7  0.7  0.7  1.0

Clearly, all approach reach the same solution except CCD. Lets check objective

In [64]:
m = 1
@show group_block_objective(Σ, equi, m, :equi)
@show group_block_objective(Σ, sdp_subopt, m, :sdp_subopt)
@show group_block_objective(Σ, sdp_ccd, m, :sdp)
@show group_block_objective(Σ, sdp_block, m, :sdp_block);

group_block_objective(Σ, equi, m, :equi) = 3.499999999999997
group_block_objective(Σ, sdp_subopt, m, :sdp_subopt) = 3.499999999999997
group_block_objective(Σ, sdp_ccd, m, :sdp) = 21.260313965235586
group_block_objective(Σ, sdp_block, m, :sdp_block) = 3.499999999999997


Why is CCD not reaching the global optimal point? It seems $S_{11}$ reached the optimal value, but why is $S_{12}$ not reaching it? Lets print some intermediate value

In [119]:
sdp_ccd, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp, robust=true);

update skipped at (2, 1), δ = 0.35, but (numerical) lower bound = 9.900166894012255e-7 and (numerical) upper bound = -9.152867099166838e-7, (mathematical lower bound) = -9.983310598774526e-9, (mathematical upper bound) = 0.2619506053079514


It seems the algorithm wanted to update $S_{2,1}^{new} = S_{2,1} + 0.35$, but it's not in a feasible region, so the update was skipped. 